# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [18]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

# Load the dataset
df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [13]:
# View the first few rows of the dataset
print(df.head())

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view


In [14]:
# Check the number of unique viewers and clickers
unique_viewers = df[df['action'] == 'view']['id'].nunique()
unique_clickers = df[df['action'] == 'click']['id'].nunique()
print("Number of unique viewers:", unique_viewers)
print("Number of unique clickers:", unique_clickers)

Number of unique viewers: 6328
Number of unique clickers: 1860


In [15]:
# Check for anomalies - anyone who clicked but didn't view
click_without_view = df[(df['action'] == 'click') & (~df['id'].isin(df[df['action'] == 'view']['id']))]
print("Number of clicks without view:", len(click_without_view))

Number of clicks without view: 0


In [16]:

# Check for overlap between control and experiment groups
overlap = len(set(df[df['group'] == 'control']['id']).intersection(set(df[df['group'] == 'experiment']['id'])))
print("Number of users in both control and experiment groups:", overlap)


Number of users in both control and experiment groups: 0


In the given dataset, there are 6,328 unique viwers and 1,860 unique clickers. 

Also indicating that all clickcers have also viewed the page.

Additinally, there are no users who appear in both the control and the experiment groups, indicating that there is no overlap between the 2 groups.



To conduct a statistical test to determine whether the experimental homepage was more effective than the control group, you can use a hypothesis test such as the chi-square test or the t-test.

For the chi-square test, you can analyze the click-through rates (CTR) for the control and experiment groups. The null hypothesis would be that there is no difference in CTR between the two groups, while the alternative hypothesis would be that the experimental homepage has a higher CTR than the control group.

For the t-test, you can compare the mean number of clicks between the control and experiment groups. The null hypothesis would be that there is no difference in the mean number of clicks, while the alternative hypothesis would be that the experimental homepage has a higher mean number of clicks than the control group.

You can choose the appropriate test based on the specific requirements and assumptions of your analysis.


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [23]:
#Your code here
# To conduct the test statistic I'll use a chi-square test.

import scipy.stats as stats

# Create a ccontingency table 
contingency_table = pd.crosstab(df['group'], df['action'])

# Perform chi-square test 
chi2, p_value, _, _ = stats.chi2_contingency(contingency_table)

# Print the results 
print("Chi-square test statistic:", chi2)
print("P-value:", p_value)




Chi-square test statistic: 3.636160051233291
P-value: 0.056537191086915774


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [25]:
# Number of viewers in the experiment group
experiment_viewers = df[df["group"] == "experiment"]["id"].nunique()

# Number of clickers in the control group
control_clickers = df[(df["group"] == "control") & (df["action"] == "click")]["id"].nunique()

# Number of viewers in the control group
control_viewers = df[df["group"] == "control"]["id"].nunique()

# Click-through rate of the control group
control_ctr = control_clickers / control_viewers

# Expected number of clicks for the experiment group
expected_clicks_experiment = experiment_viewers * control_ctr

expected_clicks_experiment


838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [26]:
#Your code here
import math

# Actual number of clicks in the experiment group
actual_clicks_experiment = df[(df["group"] == "experiment") & (df["action"] == "click")]["id"].nunique()

# Calculate the z-score
z_score = (actual_clicks_experiment - expected_clicks_experiment) / math.sqrt(expected_clicks_experiment * (1 - control_ctr))

z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [27]:
#Your code here
from scipy.stats import norm

# Calculate the p-value
p_value = 1 - norm.cdf(z_score)

p_value

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: 

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.